In [1]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

#Import dataset
time_series = pd.read_csv('dataset/time_series_after_exploratory_analysis_and_cleaning_data_1.csv', sep = ';', index_col = 0)

In [2]:
# Version that we used in this notebook
!pip install -q watermark
%reload_ext watermark
%watermark --iversions

numpy     : 1.20.3
pandas    : 1.3.4
sys       : 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
matplotlib: 3.4.3



In [3]:
#We will use line 0 to 18 to train the model, and try to predict lines 19 and 20
#add new_columns
time_series.loc[:,2016] = 0
time_series.loc[:,2017] = 0

#Predict values

for key,line in time_series.iterrows():
    if line['product'] == 'Livestock':
        poly = PolynomialFeatures(degree = 4)
    else:
        poly = PolynomialFeatures(degree = 2)
    #Adjust the model
    y = list(line.iloc[0:19])
    x = poly.fit_transform(np.array(range(0,19)).reshape(-1,1))
    line = LinearRegression()
    line.fit(x,y)

    #Predict the values
    results = line.predict(poly.fit_transform(np.array([19,20]).reshape(-1,1)))

    #Insert values into dataframe
    time_series.loc[key,2016] = 0 if results[0] < 0 else results[0]
    time_series.loc[key,2017] = 0 if results[1] < 0 else results[1]

In [4]:
#Export in the correct result
part_1 = time_series.iloc[:,21:25]
part_1.loc[:,'year'] = '2016-01-01'
part_1.reset_index(inplace = True)
part_1 = part_1.iloc[:,1:6]
part_1.columns = ['city_code', 'product', 'product_type', 'destinated_area', 'year']
part_2 = time_series.iloc[:,[21,22,23,25]].copy()
part_2.loc[:,'year'] = '2017-01-01'
part_2.reset_index(inplace = True)
part_2 = part_2.iloc[:,1:6]
part_2.columns = ['city_code', 'product', 'product_type', 'destinated_area', 'year']
df = pd.concat([part_1,part_2])
df.to_csv('8GDLJG.csv', sep = ';')

In [5]:
def wmape(time_series):
    real_part_1 = time_series.iloc[:,19].values
    real_part_2 = time_series.iloc[:,20].values
    real = np.concatenate([real_part_1,real_part_2])
    predict_part_1 = time_series.loc[:,2016].values
    predict_part_2 = time_series.loc[:,2017].values
    predict = np.concatenate([predict_part_1,predict_part_2])
    return abs(real-predict).sum()/real.sum()

In [6]:
#Calculate the error
(wmape(time_series.loc[time_series.product_type == 'pasture']) + \
wmape(time_series.loc[time_series.product_type == 'temporary']) + \
wmape(time_series.loc[time_series.product_type == 'permanent']))/3

0.15990994429535757